# INFO 2950 Fall 2025 Main Project 


In [3]:
# imported like similar to homeworks
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
import duckdb
import requests
from bs4 import BeautifulSoup
import time

## Upload/Import data

In [17]:
unemployment_df = pd.read_csv("UnemploymentData.csv", header = None)
ethno_df = pd.read_csv("The Guide to Public Opinion and Electoral Behavior.csv")

#check
print(unemployment_raw.head(15))
ethno_df.head(10)
#Note : real data starts at the 11th line of data set of first

                                                   0   \
0   Labor Force Statistics from the Current Popula...   
1                                 Original Data Value   
2                                                 NaN   
3                                          Series Id:   
4                                 Seasonally Adjusted   
5                                       Series title:   
6                                 Labor force status:   
7                                       Type of data:   
8                                                Age:   
9                                              Years:   
10                                                NaN   
11                                               Year   
12                                               1964   
13                                               1965   
14                                               1966   

                          1    2    3    4    5    6    7    8    9    10  \
0         

,Demographics,Subgroup,Total Cases,Year,Whites,Blacks,Chicanos/Hispanics,Asian-Americans
0,All,All,1532,1964,84.7,63.8,NaN,NaN
1,All,All,1273,1966,84.4,63.2,NaN,NaN
2,All,All,1503,1968,81.1,64.7,NaN,NaN
3,All,All,1469,1970,77.2,61.5,NaN,NaN
4,All,All,2120,1972,78.2,64.0,NaN,NaN
5,All,All,1525,1974,79.1,65.5,NaN,NaN
6,All,All,1869,1976,73.7,60.8,55.5,NaN
7,All,All,1382,1980,77.4,64.2,57.8,NaN
8,All,All,1321,1982,72.8,63.8,NaN,NaN
9,All,All,1902,1984,74.2,64.2,59.4,NaN


## Cleaning data for graphing the relationships


In [23]:
new_header = unemployment_df.iloc[11].tolist()
unemployment_data = unemployment_df[12:].copy()
unemployment_data.columns = new_header
unemployment_data.reset_index(drop = True, inplace = True)

if 'Year' not in unemployment_data.columns:
    unemployment_data.rename(columns={unemployment_data.columns[0]: 'Year'}, inplace = True)

#same type(numbers that we can work with)
month_cols = [col for col in unemployment_data.columns if col not in ['Year', 'Average Unemployment Rate']]
for col in month_cols:
    unemployment_data[col] = pd.to_numeric(unemployment_data[col], errors = 'coerce')

#yearly average unemployment rate
unemployment_data['Average Unemployment Rate'] = unemployment_data[month_cols].mean(axis = 1)

unemployment_data['Year'] = pd.to_numeric(unemployment_data['Year'], errors = 'coerce')

unemployment_cleaned = unemployment_data[['Year', 'Average Unemployment Rate']].dropna()

unemployment_cleaned['Year'] = unemployment_cleaned['Year'].astype(int)

#over lapping years
voting_years = [1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1996,2000, 2004, 2008, 2012, 2016, 2020, 2024]
unemplo_voting = unemployment_cleaned[unemployment_cleaned['Year'].isin(voting_years)]


unemplo_voting.head(20)

,Year,Average Unemployment Rate
0,1964,5.158333
4,1968,3.558333
8,1972,5.600000
12,1976,7.700000
16,1980,7.175000
20,1984,7.508333
24,1988,5.491667
28,1992,7.491667
32,1996,5.408333
36,2000,3.966667


In [29]:
ethno_df = pd.read_csv("The Guide to Public Opinion and Electoral Behavior.csv")

ethno_df = ethno_df[['Year', 'Whites', 'Blacks', 'Chicanos/Hispanics', 'Asian-Americans']]
ethno_avg = ethno_df.groupby('Year', as_index = False).mean(numeric_only = True)

ethno_avg.head(20)

,Year,Whites,Blacks,Chicanos/Hispanics,Asian-Americans
0,1964,82.793077,65.640000,NaN,NaN
1,1966,82.903759,64.912782,NaN,NaN
2,1968,79.858647,66.642105,NaN,NaN
3,1970,75.498519,63.234815,NaN,NaN
4,1972,76.771324,66.025000,NaN,NaN
5,1974,78.197115,66.979808,NaN,NaN
6,1976,72.956934,62.766423,56.090511,NaN
7,1980,77.412587,66.179021,59.353147,NaN
8,1982,71.793007,64.258042,NaN,NaN
9,1984,73.918919,65.443919,60.510135,NaN


In [30]:
merged_df = pd.merge(ethno_avg, unemplo_voting, on='Year', how='inner').sort_values('Year').reset_index(drop  = True)
merged_df.to_csv("merged_ethno_unemployment.csv", index = False)
# print out first couple

merged_df.head(20)

,Year,Whites,Blacks,Chicanos/Hispanics,Asian-Americans,Average Unemployment Rate
0,1964,82.793077,65.640000,NaN,NaN,5.158333
1,1968,79.858647,66.642105,NaN,NaN,3.558333
2,1972,76.771324,66.025000,NaN,NaN,5.600000
3,1976,72.956934,62.766423,56.090511,NaN,7.700000
4,1980,77.412587,66.179021,59.353147,NaN,7.175000


## Graphs 

In [1]:
print("hi")

hi
